# CNN from scratch

### Step-1: Importing and data preprocessing

In [106]:
import numpy as np
import pandas as pd

In [107]:
df = pd.read_csv(r"..\data\raw\mnist_test.csv")
df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

X = X.reshape(-1, 28, 28)
X = X/255.0

### Step-2: Define activation and loss functions

In [109]:
def relu(x):
    return np.maximum(0,x)

def softmax(x):
    e = np.exp(x - np.max(x))
    return e / np.sum(e)

def cross_entropy(pred, y):
    return -np.log(pred[y] + 1e-8)

### Step-3: Define Convolutional layer

In [110]:
class Conv:
    def __init__(self):
        self.kernel = np.random.randn(3,3)

    def forward(self, image):
        height, width = image.shape
        output = np.zeros((height - 2, width - 2))

        for i in range(height - 2):
            for j in range(width - 2):
                region = image[i:i+3, j:j+3]
                output[i][j] = np.sum(region * self.kernel)
        return output

### Step-4: Define Pooling layer

In [111]:
class Pooling:
    def forward(self, x):
        height, width = x.shape
        output = np.zeros((height//2, width//2))

        for i in range(0, height, 2):
            for j in range(0, width, 2):
                output[i//2, j//2] = np.max(x[i:i+2, j:j+2])
        return output

### Step-5: Define Neural Network layer

In [112]:
class NeuralNetwork:
    def __init__(self, input_size, output_size):
        self.W = np.random.randn(input_size, output_size) * 0.01
        self.b = np.zeros(output_size)

    def forward(self, x):
        self.x = x
        return x @ self.W + self.b
    
    def backward(self, grad, lr):
        self.W -= lr * np.outer(self.x, grad)
        self.b -= lr * grad

### Step-6: Define CNN Architecture

In [113]:
class CNN:
    def __init__(self):
        self.conv = Conv()
        self.pool = Pooling()
        self.nn = NeuralNetwork(13*13, 10)

    def forward(self, image):
        x = self.conv.forward(image)
        x = relu(x)
        x = self.pool.forward(x)
        x = x.flatten()
        x = self.nn.forward(x)
        return softmax(x)

### Step-7: Training

In [114]:
model = CNN()

learning_rate = 0.1

for i in range(1000):
    img = X[i]
    label = y[i]

    pred = model.forward(img)

    grad = pred.copy()
    grad[label] -= 1

    model.nn.backward(grad, learning_rate)

### Step-8: Testing

In [115]:
pred = model.forward(X[0])
label = y[0]


print("Actual label : ", label)
print("Predicted label : ", np.argmax(pred))

Actual label :  7
Predicted label :  7


### Step-9: Calculate accuracy

In [116]:
correct = 0

for x, y in zip(X, y):
    out = model.forward(x)
    pred = np.argmax(out)

    if pred == y:
        correct += 1

accuracy = correct / len(X)
print("Accuracy:", accuracy)


Accuracy: 0.7244
